<a href="https://colab.research.google.com/github/CarolineAndradeR/Data-Science/blob/main/analise_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conceitos**

Encontrar grupos de informação, produtos ou até mesmo pessoas com base em dados, pode ser chamada de análise de cluster ou análise de conglomerados.

Definição - Técnica de ML não supervisionada que permite agrupar dados em grupos homogêneos em função do grau de similiaridade entre os indivíduos por meio de características selecionadas.

Correlação - Coeficiente que mede a força e doreção do relacionamento linear entre duas variáveis.

Similaridade - Medida de proximidade, ou seja, deve ser comparar pares de objetos e aplicar medidas de distâncas, essas métricas verifica o quanto um indivíduo é diferente do outro.

Quando buscamos agrupar variáveis (colunas do banco de dados) utilizamos a correlação. Quando precisamos comparar indivíduos (as linhas do banco de dados) aplicamos métricas de similaridade, onde as medidas de distância serão importantes.

**Etapas para realizar uma análise de cluster**

1. Escolha das variáveis, identificação de outliers, padronização.

A seleção das variáveis é uma decisão combinada entre as áreas de negócios e Analytics. Identificação de outliers é uma etapa importante para encontrar nos dados valores acima ou abaixo do considerado normal.
A tarefa mais importante é a padronização das variáveis, em diversos momentos, as colunas possuem unidades de medidas diferentes e nesse caso, não podemos aplicar o algoritmo diretamente, precisamos ajustar esses dados.

2. MEDIDA DE DISTÂNCIA (SIMILARIDADE).

Aaplicar a medida de distância, a métrica mais utilizada em análise de cluster é a distância euclidiana. Essa métrica é uma generalização do teorema de Pitágoras.

3. ESCOLHA DO ALGORITMO DE AGRUPAMENTO (HIERÁRQUICOS E NÃO HIERÁRQUICOS).

Buscar uma forma de organizar cada indivíduo no seu respectivo grupo. Definição do método de agrupamento. Existe 2 gerais, hierárquicos e não hierárquicos e o método híbrido que é a combinação dos dois. Os algoritmos hierárquicos geram os grupos de forma iterativa, em cada etapa, é vista a distâncias entre os múltiplos indivíduos e as menores distâncias são agrupadas formando os clusters e no final, um coeficiente de aglomeração é obtido informando em cada iteração como os grupos foram ajustados.
E no caso dos não hierárquicos, é preciso definir a quantidade de grupos no inicio e após isso, a cada iteração as distâncias são obtidas e os grupos são formados.

4. ESCOLHA DA QUANTIDADE DE GRUPOS.

Nos métodos hierárquicos quanto nos algoritmos não hierárquicos existe técnicas para obter o número ideal de clusters. Não tem um padrão sobre quais as melhores ou mais eficientes, porém adota-se os hierárquicos para auxiliar no processo de obter esse número ideal.

5. INTERPRETAÇÃO DOS GRUPOS

Interpretar os grupos propostos pela análise. Uma vez que os algoritmos de aglomeração são não supervisionados, não tem uma medida direta para avaliar se a solução apresentada foi assertiva ou não. Uma alinhamento com a equipe de negócios e analistas deve ser organizada. Traduzir os resultados é a criação de personas que representam as características gerais dos grupos, esses resultado pode combinar os grupos formados pelas variáveis intervalares e combinar com variáveis qualitativas.

# **Prática**

In [ ]:
# gerais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# cluster
import scipy.cluster.hierarchy as shc
from sklearn.preprocessing import MinMaxScaler, scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

# carregar dados
dados = pd.read_csv('rfm_analysis.csv')

In [ ]:
# dados rfm
dados.head()

In [ ]:
# selecionar colunas
cols = ['Recency', 'Frequency', 'Monetary_Value']
dados_cluster = dados[cols]

# min-max
minmax_scaler = MinMaxScaler()
dados_minmax = minmax_scaler.fit_transform(dados_cluster)
dados_minmax.shape

# normalizar
dados_normalizados = scale(dados_cluster)
dados_normalizados.shape

In [ ]:
# min-max dataframe
dados_minmax = pd.DataFrame(dados_minmax, columns=dados_cluster.columns)
dados_minmax['CustomerID'] = dados['CustomerID']
dados_minmax.head()

# scale dataframe
dados_normalizados = pd.DataFrame(dados_normalizados, columns=dados_cluster.columns)
dados_normalizados['CustomerID'] = dados['CustomerID']
dados_normalizados.head()

In [ ]:
# algoritmo hierarquico
dados_cluster_hierarquico = dados_normalizados[cols].sample(300, random_state = 5461)
cluster_hier = shc.linkage(dados_cluster_hierarquico, method='ward')
# grafico
plt.figure(figsize=(10, 7))
plt.title("Dendograma")
dend = shc.dendrogram(cluster_hier)
plt.axhline(y=10, color='r', linestyle='--')

In [ ]:
# algoritmo hierarquico
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward', compute_distances=True)
cluster.fit(dados_cluster_hierarquico)
distancias_grupos = pd.DataFrame(cluster.distances_, columns = ['distancia'])
distancias_grupos['iteracao'] = distancias_grupos.index
distancias_grupos['qtd_grupos'] = (max(distancias_grupos['iteracao']) - distancias_grupos['iteracao'])+1

# grafico
n_cluster_test = 15
avaliacao_clusters = distancias_grupos[distancias_grupos['qtd_grupos'] <= n_cluster_test][['distancia', 'qtd_grupos']]
plt.plot(avaliacao_clusters['qtd_grupos'], avaliacao_clusters['distancia'], '-o')
plt.show()

In [ ]:
# obter grupos
dados_cluster_hierarquico['cluster'] = cluster.fit_predict(dados_cluster_hierarquico)

# centroides
centroides = dados_cluster_hierarquico.groupby('cluster')                      .agg(media_recencia = pd.NamedAgg('Recency', 'mean'),
                          media_frequencia = pd.NamedAgg('Frequency', 'mean'),
                          media_valor = pd.NamedAgg('Monetary_Value', 'mean'))
centroides

In [ ]:
# treinamento
cluster_kmeans = KMeans(n_clusters = 3, init = centroides.values, n_init=1, max_iter = 1000)
cluster_kmeans.fit(dados_normalizados[cols])
# associar os clusters aos dados
dados['cluster'] = cluster_kmeans.predict(dados_normalizados[cols])
dados.head()

In [ ]:
# contagem
dados.groupby('cluster').size().to_frame('n')

In [ ]:
# centroides dos grupos
centroides_final = dados.groupby('cluster')      .agg(media_recencia = pd.NamedAgg('Recency', 'mean'),
          media_frequencia = pd.NamedAgg('Frequency', 'mean'),
          media_valor = pd.NamedAgg('Monetary_Value', 'mean'))
# medias gerais
media_recencia = np.mean(centroides_final['media_recencia'])
media_frequencia = np.mean(centroides_final['media_frequencia'])
media_valor = np.mean(centroides_final['media_valor'])
# regras de avaliacao
centroides_ajustado = centroides_final.copy()
centroides_ajustado['media_recencia'] = np.where(centroides_final['media_recencia'] > media_recencia, 'alto', 'baixo')
centroides_ajustado['media_frequencia'] = np.where(centroides_final['media_frequencia'] > media_frequencia, 'alto', 'baixo')
centroides_ajustado['media_valor'] = np.where(centroides_final['media_valor'] > media_valor, 'alto', 'baixo')
# ver os resultados
centroides_ajustado